## Modeling rain accretion

### Define reference variables

In [9]:
using QuadGK
using SpecialFunctions

In [98]:
rho_w = 998
rho_a = 1
g = 9.81
C_d = 0.55
m_e = 3
r_0 = 1e-3
r_0 = 1e-3
n_0 = 16e6
q_c = range(1e-8, 5e-3, 100)
E_cp = 0.8

0.8

In [99]:
m_0 = 4*pi*rho_w*r_0*r_0*r_0/3
a_0 = pi*r_0*r_0
v_0 = sqrt((8*g*r_0*(rho_w - rho_a))/(3*C_d*rho_a))
# lambda = ((gamma(m_e+1)*m_0*n_0)/(q_c*rho_w*(r_0^m_e))).^(1/(m_e+1))

6.886286768247852

In [100]:
f(r) = [exp(-(((gamma(m_e+1)*m_0*n_0)/(q*rho_w*(r_0^m_e))).^(1/(m_e+1)))*r)*r^(5/2) for q in q_c]

f (generic function with 1 method)

In [101]:
I,err = quadgk(f, 0, Inf, rtol=1e-8)

([9.83469912415687e-15, 1.7110148507259943e-11, 3.137743293492319e-11, 4.473884031282986e-11, 5.75439734887408e-11, 6.995074539247136e-11, 8.20490258255749e-11, 9.389664579103598e-11, 1.0553382729360304e-10, 1.16990096905304e-10  …  8.772867632334086e-10, 8.858100228550897e-10, 8.943215825153481e-10, 9.028215851925636e-10, 9.113101706001526e-10, 9.19787475295045e-10, 9.282536327814572e-10, 9.367087736102023e-10, 9.45153025473788e-10, 9.53586513297507e-10], 4.5299919633020224e-17)

In [102]:
dq_dt = n_0*a_0*v_0*E_cp*(r_0^(5/2))*I

100-element Vector{Float64}:
 8.612039726944e-20
 1.4982998139362686e-16
 2.7476559837131356e-16
 3.9176864004421255e-16
 5.039005052162523e-16
 6.125440042894589e-16
 7.184861082646319e-16
 8.22233231104074e-16
 9.241375884662595e-16
 1.0244577383488399e-15
 1.123391194788697e-15
 1.2210935325636104e-15
 1.3176903086938777e-15
 ⋮
 7.532630850215146e-15
 7.607476030540501e-15
 7.682216162902636e-15
 7.756852559541022e-15
 7.831386502071452e-15
 7.90581924252561e-15
 7.980152004344575e-15
 8.054385983328738e-15
 8.128522348546516e-15
 8.202562243203998e-15
 8.276506785477684e-15
 8.350357069312161e-15